## This is for generating images from the diffusion model

In [1]:
import numpy as np
import pandas as pd
import os
import h5py
import cv2
import matplotlib.pyplot as plt
import tqdm
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
# current_dir = "/Users/thatblue340/Documents/Documents/GitHub/EECS-545-final-project"
current_dir = ""

In [2]:
from denoising_diffusion_pytorch import Unet, GaussianDiffusion

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Define the U-Net model
model = Unet(
    dim=128,
    dim_mults=(1, 2, 4, 8),
    channels=3
)

# Define the diffusion wrapper
diffusion = GaussianDiffusion(
    model,
    image_size=128,
    timesteps=1000,
    objective='pred_x0'
)

In [5]:
model.to(device)
diffusion.to(device)

GaussianDiffusion(
  (model): Unet(
    (init_conv): Conv2d(3, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (time_mlp): Sequential(
      (0): SinusoidalPosEmb()
      (1): Linear(in_features=128, out_features=512, bias=True)
      (2): GELU(approximate='none')
      (3): Linear(in_features=512, out_features=512, bias=True)
    )
    (downs): ModuleList(
      (0): ModuleList(
        (0-1): 2 x ResnetBlock(
          (mlp): Sequential(
            (0): SiLU()
            (1): Linear(in_features=512, out_features=256, bias=True)
          )
          (block1): Block(
            (proj): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm): RMSNorm()
            (act): SiLU()
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (block2): Block(
            (proj): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm): RMSNorm()
            (act): SiLU()
            (dropout): Dropout(

In [9]:
checkpoint = torch.load('saved_models/diffusion_checkpoint_7500.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
diffusion.load_state_dict(checkpoint['diffusion_state_dict'])

<All keys matched successfully>

In [10]:
model.eval()
diffusion.eval()   

GaussianDiffusion(
  (model): Unet(
    (init_conv): Conv2d(3, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (time_mlp): Sequential(
      (0): SinusoidalPosEmb()
      (1): Linear(in_features=128, out_features=512, bias=True)
      (2): GELU(approximate='none')
      (3): Linear(in_features=512, out_features=512, bias=True)
    )
    (downs): ModuleList(
      (0): ModuleList(
        (0-1): 2 x ResnetBlock(
          (mlp): Sequential(
            (0): SiLU()
            (1): Linear(in_features=512, out_features=256, bias=True)
          )
          (block1): Block(
            (proj): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm): RMSNorm()
            (act): SiLU()
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (block2): Block(
            (proj): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm): RMSNorm()
            (act): SiLU()
            (dropout): Dropout(

In [11]:
# %pip install ipywidgets==8.0.7

In [12]:
from tqdm import tqdm  # CLI-safe version
from torchvision.transforms.functional import to_pil_image

# Create output directory if it doesn't exist
os.makedirs("images", exist_ok=True)

# Generate 10 images
with torch.no_grad():
    samples = diffusion.sample(batch_size=10)

# Denormalize: [-1, 1] → [0, 1]
samples = (samples + 1) / 2
samples = samples.clamp(0, 1).cpu()

# Save each image
for i in range(samples.size(0)):
    img = to_pil_image(samples[i])
    img.save(f"images/sample_{i:03}.png")

print("✅ Saved all images to the 'images/' folder.")

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

✅ Saved all images to the 'images/' folder.
